In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Define parameters for goodFeaturesToTrack
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

# Initialize YOLO model
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")  # Make sure these paths are correct
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Open the video
cap = cv2.VideoCapture("video.mp4")

# Initialize the previous frame and points for optical flow
prev_gray = None
prev_points = None

# Loop through the video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect objects using YOLO
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Get the bounding boxes of detected objects
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Minimum confidence threshold
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                w = int(detection[2] * frame.shape[1])
                h = int(detection[3] * frame.shape[0])
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])

    # If previous points exist, calculate optical flow
    if prev_points is not None:
        # Calculate optical flow
        next_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_points, None)

        # Ensure status array matches points
        valid_points = next_points[status == 1]
        prev_valid_points = prev_points[status == 1]

        # Draw lines between matching points
        for i, (new, old) in enumerate(zip(valid_points, prev_valid_points)):
            a, b = new.ravel().astype(int)  # Ensure points are integers
            c, d = old.ravel().astype(int)  # Ensure points are integers
            frame = cv2.line(frame, (a, b), (c, d), (0, 255, 0), 2)

        # Now, calculate homography if enough points are valid
        if len(valid_points) >= 4:  # At least 4 points are needed for homography
            H, _ = cv2.findHomography(prev_valid_points, valid_points, cv2.RANSAC)
            # Optionally, you can warp the frame with the homography matrix
            height, width = frame.shape[:2]
            warped_frame = cv2.warpPerspective(frame, H, (width, height))

    # Detect new feature points
    prev_points = cv2.goodFeaturesToTrack(gray, mask=None, **feature_params)

    # Show the frame with detected optical flow
    cv2.imshow('Frame', frame)

    # Press 'q' to quit the video display window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Update previous frame
    prev_gray = gray

# Release resources and close windows
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    cv2.imshow("Video", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break

cap.release()
cv2.destroyAllWindows()
